# 🦙 Fully Open-Source RAG in Google Colab


In [ ]:
# Uninstall the current PyTorch installation
!pip uninstall torch -y

# Install PyTorch with CUDA 12.1 compatibility
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 545.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━

In [ ]:
# Install required libraries
!pip install -q llama-index-core llama-index-embeddings-huggingface \
                 llama-index-vector-stores-faiss transformers accelerate \
                 torch sentencepiece bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
!pip install faiss-gpu-cu12[fix-cuda]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you 

In [ ]:
# testing number of GPU's assign
import faiss
print(faiss.get_num_gpus())

1


In [ ]:

# 1️⃣ Imports
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
# 2️⃣ Load your local data (put some .txt files in /content/data)
data_path = "/content/data"
documents = SimpleDirectoryReader(data_path).load_data()

In [ ]:
documents

[Document(id_='3585ff15-7d76-4b41-869c-6efeecdc3d08', embedding=None, metadata={'file_path': '/content/data/Resume-Tushar-Wagh-8-yrs-ML-Data-Research-txt.txt', 'file_name': 'Resume-Tushar-Wagh-8-yrs-ML-Data-Research-txt.txt', 'file_type': 'text/plain', 'file_size': 3772, 'creation_date': '2025-10-07', 'last_modified_date': '2025-10-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Contact\r\nIndia\r\n9890132816 (Mobile)\r\nwaghts95@gmail.com\r\nwww.linkedin.com/in/tushar-wagh-1781b7112 (LinkedIn)\r\ngithub.com/waghts95 (Other)\r\nTop Skills\r\nPython\r\nData Science\r\nCloud Management\r\nTushar Wagh\r\nData & MLOps Engineer\r\nPune, M

In [ ]:
# 3️⃣ Local embedding model (Hugging Face)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 4️⃣ Local FAISS vector store
import faiss

# Corrected initialization: Create a FAISS index first
dimension = 384 # This should match the dimension of your embeddings
faiss_index = faiss.IndexFlatL2(dimension) # Example: using L2 distance

faiss_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=faiss_store)

In [ ]:
# 5️⃣ Build the vector index
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

In [ ]:
from huggingface_hub import login

# @title Login to Hugging Face
# @markdown You need to accept the terms and conditions of the model on the Hugging Face website first.
# @markdown You can get a token from your settings page: https://huggingface.co/settings/tokens
try:
  from google.colab import userdata
  HF_TOKEN = userdata.get('HF_TOKEN')
except:
  HF_TOKEN = input("Please enter your Hugging Face token: ")

login(token=HF_TOKEN)

In [ ]:
# 6️⃣ Load an open-source LLM (via transformers)
# Recommended: small instruct model to fit in Colab GPU
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True  # use less VRAM
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# 7️⃣ Query + Generate function (RAG)
def generate_response(query: str):
    # Retrieve top-K context chunks
    retriever = index.as_retriever(similarity_top_k=3)
    retrieved_docs = retriever.retrieve(query)
    context_text = "\n\n".join([d.get_text() for d in retrieved_docs])

    # Build final prompt for LLM
    prompt = (
        f"Context:\n{context_text}\n\n"
        f"Question: {query}\n\n"
        f"Answer concisely using the context above."
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# 8️⃣ Example usage
query = "how many years experience does Tushar Wagh has ?"
print(generate_response(query))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Context:
Contact
India
9890132816 (Mobile)
waghts95@gmail.com
www.linkedin.com/in/tushar-wagh-1781b7112 (LinkedIn)
github.com/waghts95 (Other)
Top Skills
Python
Data Science
Cloud Management
Tushar Wagh
Data & MLOps Engineer
Pune, Maharashtra, India
Summary
As a leader,I believe that every brain is unique.I believe in 1. Fostering a culture where vulnerability is accepted.2. Growth does not mean to win everytime, important is you walkhand-in-hand.I firmly,Encourage and respect: Ideas, creativity, art and innovation.Believe in : "Take your time and get back with something never seenbefore"Technical skills : Data Science : Machine Learning and Deep Learning algorithms forComputer Vision and LLM.Data Engineering: Data Pipelines and Data ProfilingTools and Technologies:1. Python : Have hands on Deep learning, Machine Learningpackages, Image processing packages, Data Processing packages,Numerical calculation packages, Data Visualization packages, DataQuality packages.2. AWS : AWS Data pipel